<a href="https://colab.research.google.com/github/MartinSantaGitHub/python-ml-course/blob/master/code/T11%20-%201%20-%20TensorFlow101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%tensorflow_version 1.x

TensorFlow is already loaded. Please restart the runtime to change versions.


# **Introducción a Tensor Flow**

In [1]:
import tensorflow as tf

In [2]:
print(tensorflow.__version__)

NameError: ignored

In [ ]:
x1 = tf.constant([1,2,3,4,5])
x2 = tf.constant([6,7,8,9,10])